In [1]:
import solution_file_processing as sfp
import os
import numpy as np
import pandas as pd
from dask import dataframe as dd


from solution_file_processing.utils.utils import catch_errors
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE
from solution_file_processing.constants import VRE_TECHS
from solution_file_processing.timeseries import create_output_11 as create_ts_output_11
from solution_file_processing.timeseries import create_output_4 as create_timeseries_output_4
from solution_file_processing import log
from solution_file_processing.plots import _get_plot_1_variables

# Initialize config with toml file
c = sfp.SolutionFilesConfig('config_archive/ukraine/UKR.toml')

from solution_file_processing.plots import create_plot_1a

[10:47:10 00:00] I:117:__init__ - Logging to C:\Users\hart_c\showcase\solution-file-processing\logs\20240319_104710-UKR.log.
[10:47:11 00:00] I:149:__init__ - Initialized SolutionFilesConfig for config_archive/ukraine/UKR.toml.


In [4]:
c.v.gen_op_costs_by_reg.groupby(["model", "property"]).sum().unstack("model").fillna(0)

In [ ]:
reg_ids

In [ ]:

_, _,_,gen_stack_by_reg = _get_plot_1_variables(c)



In [ ]:
c.v.exports_ts.abs()

In [ ]:
en_stack_total[c.GEO_COLS] = 'Overall'
gen_stack_total = gen_stack_total.set_index(['model'] + c.GEO_COLS + ['timestamp'])
    

exports_total[c.GEO_COLS] = 'Overall'
exports_total = exports_total.set_index(['model'] + c.GEO_COLS + ['timestamp'])
imports_total = imports_total.set_index(['model'] + c.GEO_COLS + ['timestamp'])


gen_stack_total.loc[pd.IndexSlice[:,'Overall', 'Overall', :],'Imports'] = imports_total.values
gen_stack_total.loc[pd.IndexSlice[:,'Overall', 'Overall', :],'Net Load w/ Exports'] = \
    gen_stack_total.loc[pd.IndexSlice[:,'Overall', 'Overall', :],'Net Load w/ Exports'].values + exports_total.values


In [ ]:
imports_total

Stuff for validation .... 

In [ ]:
try:
    c.v.line_imp_exp.droplevel('regZo')
except KeyError:
    c.v.line_imp_exp

In [ ]:
### Split hourly demand using split in snapshots
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE
import openpyxl as xl 
  
modelled_regs = ['Ukraine', 'CR', 'ER', 'NR', 'SR', 'SWR', 'SR', 'EOR']
demand_sector_dict = {}
demand_by_reg_df = pd.DataFrame(None)

folder_path = 'Y:/Modelling/Ukraine/2023_UKR_ST_Security/01_Data/02_Demand'
demand_file_paths = [f for f in os.listdir(folder_path) if ('demand' in f) &('$' not in f)]

# for path in demand_file_paths:
for path in demand_file_paths:

    full_path = os.path.join(folder_path, path)
    # load the workbook 
    wb = xl.load_workbook(full_path)     
    sheet_names = wb.sheetnames
    est_gen_by_tech_df = {}

    for reg in sheet_names:

        aux_loss_df = pd.read_excel(full_path, sheet_name=reg, header=7, usecols='A:Z', nrows = 52)
        
        aux_loss_df = aux_loss_df.loc[aux_loss_df['consumer groups'].str.contains('needs'),:]
        aux_loss_df = aux_loss_df.iloc[1:,]

        rename_cols = {"The CHP's own needs":"CHP",
                            "Own needs of HPP": "Hydro",
                            "The NPP's own needs":"Nuclear",
                            "Production needs":"Thermal"}
        
        aux_loss_df['consumer groups'] = aux_loss_df['consumer groups'].apply(lambda x: rename_cols[x])
        aux_loss_df = aux_loss_df.rename(columns={'consumer groups':'Tech'}).set_index('Tech').drop(columns='line').T.rename_axis('Hour')

            
        aux_losses_by_tech = pd.Series({'CHP':0.05, ## Gas is 4%, coal is 6% 
                            'Hydro':0.01,
                            'Nuclear':0.06,
                            'Thermal':0.05})
        
        tech_palette = {'CHP':'grey20', ## Gas is 4%, coal is 6% 
                            'Hydro':'bl',
                            'Nuclear':'p',
                            'Thermal':'brown'}
        
        est_gen_by_tech_df[reg] = aux_loss_df/aux_losses_by_tech

        with pd.ExcelWriter(os.path.join(folder_path,'est_gen_snapshot_{}.xlsx'.format(path[-13:])),
                            engine="xlsxwriter") as writer:
            # ExcelWriter for some reason uses writer.sheets to access the sheet.
            # If you leave it empty it will not know that sheet Main is already there
            # and will create a new sheet.

            for reg in est_gen_by_tech_df.keys():
                write_xlsx_stack(df=est_gen_by_tech_df[reg],
                                    writer=writer,
                                    sheet_name=reg,
                                    palette=tech_palette)
                print(f'Created sheet "{reg}" in {folder_path}.')
        


In [ ]:
c.v.gen_by_tech_ts.head()